In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import tensorflow as tf
from imblearn.under_sampling import TomekLinks
from keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/csv_result-cm1.csv')

In [4]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
tl = TomekLinks()
X_train_resampled, y_train_resampled = tl.fit_resample(X_train, y_train)

In [7]:
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [8]:
model = Sequential()
model.add(Dense(64, input_dim=X_train_resampled.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [9]:
model.compile(loss=focal_loss(), optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [10]:
model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_split=0.1, verbose=2)

Epoch 1/10
11/11 - 2s - loss: nan - accuracy: 0.8275 - val_loss: nan - val_accuracy: 0.8947 - 2s/epoch - 225ms/step
Epoch 2/10
11/11 - 0s - loss: nan - accuracy: 0.9035 - val_loss: nan - val_accuracy: 0.8947 - 69ms/epoch - 6ms/step
Epoch 3/10
11/11 - 0s - loss: nan - accuracy: 0.9035 - val_loss: nan - val_accuracy: 0.8947 - 48ms/epoch - 4ms/step
Epoch 4/10
11/11 - 0s - loss: nan - accuracy: 0.9035 - val_loss: nan - val_accuracy: 0.8947 - 55ms/epoch - 5ms/step
Epoch 5/10
11/11 - 0s - loss: nan - accuracy: 0.9035 - val_loss: nan - val_accuracy: 0.8947 - 56ms/epoch - 5ms/step
Epoch 6/10
11/11 - 0s - loss: nan - accuracy: 0.9035 - val_loss: nan - val_accuracy: 0.8947 - 53ms/epoch - 5ms/step
Epoch 7/10
11/11 - 0s - loss: nan - accuracy: 0.9035 - val_loss: nan - val_accuracy: 0.8947 - 75ms/epoch - 7ms/step
Epoch 8/10
11/11 - 0s - loss: nan - accuracy: 0.9035 - val_loss: nan - val_accuracy: 0.8947 - 54ms/epoch - 5ms/step
Epoch 9/10
11/11 - 0s - loss: nan - accuracy: 0.9035 - val_loss: nan - v

In [11]:
y_pred = model.predict(X_test)
y_pred_bin = (y_pred > 0.5).astype(int)

4/4 [==============================] - 0s 4ms/step


In [12]:
accuracy = accuracy_score(y_test, y_pred_bin)
print("Accuracy:", accuracy)

Accuracy: 0.88


In [13]:
threshold = 0.5
y_pred_bin = (y_pred > threshold).astype(int)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_bin))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_bin))

Confusion Matrix:
[[88  0]
 [12  0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94        88
           1       0.00      0.00      0.00        12

    accuracy                           0.88       100
   macro avg       0.44      0.50      0.47       100
weighted avg       0.77      0.88      0.82       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Train your machine learning model and make predictions on the test data
# Replace 'model' with your trained model and 'X_test' and 'y_test' with your test data

# Example:
# model.fit(X_train_oversampled, y_train_oversampled)
# y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred_bin)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2)
r2 = r2_score(y_test, y_pred_bin)

# Calculate Adjusted R-squared (Adj R2)
n = len(X_test)  # Number of samples
p = X_test.shape[1]  # Number of features
adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

# Print the results
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)
print("Adjusted R-squared (Adj R2):", adj_r2)

Mean Squared Error (MSE): 0.12
Root Mean Squared Error (RMSE): 0.34641016151377546
R-squared (R2): -0.13636363636363646
Adjusted R-squared (Adj R2): -0.46103896103896136
